## Prompt Chaining

In [19]:
from langgraph.graph import StateGraph, START, END
from langchain_ollama import ChatOllama
from typing import TypedDict


In [20]:
model = ChatOllama(
    model="llama3.2:latest"
)

In [3]:
model.invoke("Hello")

AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-12-18T11:31:27.293439011Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4635459927, 'load_duration': 4484349684, 'prompt_eval_count': 26, 'prompt_eval_duration': 44232996, 'eval_count': 8, 'eval_duration': 99791300, 'logprobs': None, 'model_name': 'llama3.2:latest', 'model_provider': 'ollama'}, id='lc_run--019b313a-999f-76f1-9963-10687520e698-0', usage_metadata={'input_tokens': 26, 'output_tokens': 8, 'total_tokens': 34})

In [21]:
# creating graph state
class BlogState(TypedDict):
    title: str
    outline: str
    content: str


In [27]:
def outline_gen(state: BlogState) -> BlogState:
    """ will take title and generate outline for blog"""
    prompt = f"Generate a outline of around 100 words for the given title. \n title: {state['title']}"

    result = model.invoke(prompt).content
    state['outline'] = result

    return state


In [28]:
def blog_gen(state: BlogState) -> BlogState:
    """will take outline and generate blog content"""
    prompt = f"From the given blog outline generate a blog content. \n outline: {state['outline']}"
    result = model.invoke(prompt).content
    state['content'] = result
    return state

In [29]:
graph = StateGraph(BlogState)

# defining nodes
graph.add_node("outline_gen", outline_gen)
graph.add_node("blog_gen", blog_gen)

# defining edges
graph.add_edge(START, "outline_gen")
graph.add_edge('outline_gen', "blog_gen")
graph.add_edge( "outline_gen", END)


#compling
workflow = graph.compile()



In [32]:
initial_state = {"title":"AI in Nepal"}
final_state = workflow.invoke(initial_state)

In [33]:
print(final_state)

{'title': 'AI in Nepal', 'outline': 'Here is a 100-word outline on "AI in Nepal":\n\nI. Introduction\n    A. Overview of Artificial Intelligence (AI) and its applications\n\nII. Current State of AI in Nepal\n    A. Government initiatives to promote AI adoption\n    B. Private sector investments in AI-powered solutions\n    C. Challenges faced by the Nepalese tech industry\n\nIII. Applications of AI in Nepal\n    A. Healthcare: digital health record systems, disease diagnosis, and telemedicine\n    B. Education: AI-powered adaptive learning platforms, language translation, and content creation\n    C. Agriculture: precision farming, crop monitoring, and soil analysis\n\nIV. Future Directions\n    A. Government policies to support AI innovation and entrepreneurship\n    B. Potential for Nepal to become an AI hub in South Asia', 'content': '**Unlocking the Potential of Artificial Intelligence in Nepal**\n\nArtificial intelligence (AI) has been gaining momentum globally, transforming vario